In [1]:
%load_ext autoreload
%autoreload 1
import pandas as pd
import os
from abc import ABC
from gensim.corpora import Dictionary
from torch.utils.data import Dataset
import gensim.downloader as api
from time import time
import gensim
import gensim.downloader as api
from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
from argparse import Namespace

import numpy as np
import json

from Dataset import JobDescriptionDataset
from Vectorizer import Word2vecVectoriser, OneHotVectoriser, PretrainedVectoriser

%aimport Dataset
%aimport Vectorizer

#from 'Dataset' import JobDescriptionDataset

from gensim.models import Word2Vec
from nltk.corpus import stopwords
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm



In [2]:
# df, train, test,val = load_data("task2", send_computer_to_hell=False)
# df

## Utilities

### Batch Generator

In [3]:
def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will
    ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

### Training Utils

In [4]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
        
def load_glove_from_file(glove_filepath):
    """
    Load the GloVe embeddings 
    
    Args:
        glove_filepath (str): path to the glove embeddings file 
    Returns:
        word_to_index (dict), embeddings (numpy.ndarary)
    """

    word_to_index = {}
    embeddings = []
    with open(glove_filepath, encoding="utf8") as fp:
        for index, line in enumerate(fp):
            line = line.split(" ") # each line: word num1 num2 ...
            word_to_index[line[0]] = index # word = line[0] 
            embedding_i = np.array([float(val) for val in line[1:]])
            embeddings.append(embedding_i)
    return word_to_index, np.stack(embeddings)

def make_embedding_matrix(glove_filepath, words):
    """
    Create embedding matrix for a specific set of words.
    
    Args:
        glove_filepath (str): file path to the glove embeddigns
        words (list): list of words in the dataset
    """
    word_to_idx, glove_embeddings = load_glove_from_file(glove_filepath)
    embedding_size = glove_embeddings.shape[1]
    
    final_embeddings = np.zeros((len(words), embedding_size))

    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = glove_embeddings[word_to_idx[word]]
        else:
            embedding_i = torch.ones(1, embedding_size)
            torch.nn.init.xavier_uniform_(embedding_i)
            final_embeddings[i, :] = embedding_i

    return final_embeddings

## Vectorisers

### Word2Vec

In [5]:
def test_word2vec():
    df, train, test,val = load_data("task1", send_computer_to_hell=False)

    v = Word2vecVectoriser.from_dataframe(df)
    print(v.num_features)
    print(v.vectorize(df.tfidf10[0]))

    v = Word2vecVectoriser.from_dataframe(df, data="job_description", cutoff=0)
    print(v.num_features)
    print(v.vectorize(df.job_description[0]))
    

### One Hot

In [6]:
def test_onehot():
    df, train, test,val = load_data("task1", send_computer_to_hell=False)

    v = OneHotVectoriser.from_dataframe(df, is_sequence=False, data_field="tfidf10", feature_field="is_fulltime", sent_embed=False)
    print(v.num_features)
    print(v.vectorize(df.tfidf10[0]).shape)

    v = OneHotVectoriser.from_dataframe(df, is_sequence=False, data_field="tfidf10", feature_field="is_fulltime", sent_embed=True)
    print(v.num_features)
    print(v.vectorize(df.tfidf10[0]).shape)

    v = OneHotVectoriser.from_dataframe(df, is_sequence=True, data_field="tfidf10", feature_field="is_fulltime", sent_embed=False)
    print(v.num_features)
    print(v.vectorize(df.job_description[0]).shape)
    
    v = OneHotVectoriser.from_dataframe(df, is_sequence=True, data_field="job_description", feature_field="is_fulltime", sent_embed=False)
    print(v.num_features)
    print(v.vectorize(df.job_description[0]).shape)

    v = OneHotVectoriser.from_dataframe(df, is_sequence=True, data_field="job_description", feature_field="is_fulltime", sent_embed=True)
    print(v.num_features)
    print(v.vectorize(df.job_description[0]).shape)

# test_onehot()

# Binary Classification

## Models

### Feed Forward Neural Network

In [7]:
class JobDescriptionNotSimpleClassifier(nn.Module):
    """ a simple perceptron-based classifier """
    def __init__(self, num_features):
        """
        Args:
            num_features (int): the size of the input feature vector
        """
        super(JobDescriptionNotSimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, out_features=int(num_features/2))
        self.fc2 = nn.Linear(in_features=int(num_features/2), out_features=1)
    
    def forward(self, x_in, apply_sigmoid=False):
        """The forward pass of the classifier
        Args:
            x_in (torch.Tensor): an input data tensor 
                    x_in.shape should be (batch, num_features)
            apply_sigmoid (bool): a flag for the sigmoid activation
                    should be false if used with the cross-entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch,).
        """
        x_in = self.fc1(x_in)
        x_in = F.relu(x_in)
        y_out = self.fc2(x_in).squeeze()
        
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)
            
        return y_out

In [8]:
class JobDescriptionSimpleClassifier(nn.Module):
    """ a simple perceptron-based classifier """
    def __init__(self, num_features):
        """
        Args:
            num_features (int): the size of the input feature vector
        """
        super(JobDescriptionSimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, out_features=1)
    
    def forward(self, x_in, apply_sigmoid=False):
        """The forward pass of the classifier
        Args:
            x_in (torch.Tensor): an input data tensor 
                    x_in.shape should be (batch, num_features)
            apply_sigmoid (bool): a flag for the sigmoid activation
                    should be false if used with the cross-entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch,).
        """
        y_out = self.fc1(x_in).squeeze()
        
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)
        return y_out

#### Training

In [9]:
def do_FWD_train(args):
    def make_train_state(args):
        return {'epoch_index': 0,
                'train_loss': [],
                'train_acc': [],
                'val_loss': [],
                'val_acc': [],
                'test_loss': 1,
                'test_acc': 1}

    train_state = make_train_state(args)
    if not torch.cuda.is_available():
        args.cuda = False
    args.device = torch.device("cuda" if args.cuda else "cpu")

    # dataset and vectorizer
    # dataset2 = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.input_name, Word2vecVectoriser)
    # vectorizer2 = dataset2.get_vectorizer()
    dataset = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.name, args.vectoriser, args.data_field, args.feature_field, args.is_sequence, args.sent_embed)
    vectorizer = dataset.get_vectorizer()
    print(vectorizer)
    # model
    # classifier = JobDescriptionClassifier(num_features=len(vectorizer.input_vocab))
    classifier = JobDescriptionNotSimpleClassifier(num_features=vectorizer.num_features)
    classifier = classifier.to(args.device)
    # loss and optimizer
    loss_func = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)

    def compute_accuracy(y_pred, y_target):
        y_target = y_target.cpu()
        y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()#.max(dim=1)[1]
        n_correct = torch.eq(y_pred_indices, y_target).sum().item()
        return n_correct / len(y_pred_indices) * 100
    
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index
        # Iterate over training dataset
        # setup: batch generator, set loss and acc to 0, set train mode on
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()
        for batch_index, batch_dict in tqdm(enumerate(batch_generator), colour='white'):
            # the training routine is 5 steps:
            # step 1. zero the gradients
            optimizer.zero_grad()
            # step 2. compute the output
            y_pred = classifier(x_in=batch_dict['x_data'].float())
            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_batch = loss.item()
            running_loss += (loss_batch-running_loss) / (batch_index + 1)
            # step 4. use loss to produce gradients
            loss.backward()
            # step 5. use optimizer to take gradient step
            optimizer.step()
            # compute the accuracy
            acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_batch - running_acc) / (batch_index + 1)

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset
        # setup: batch generator, set loss and acc to 0, set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            # step 1. compute the output
            y_pred = classifier(x_in=batch_dict['x_data'].float())
            # step 2. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_batch = loss.item()
            running_loss += (loss_batch - running_loss) / (batch_index + 1)
            # step 3. compute the accuracy
            acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_batch - running_acc) / (batch_index + 1)
        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)
        
    dataset.set_split('test')
    batch_generator = generate_batches(dataset,batch_size=args.batch_size,device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()
    for batch_index, batch_dict in enumerate(batch_generator):
        # compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch - running_loss) / (batch_index + 1)
        # compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)
    train_state['test_loss'] = running_loss
    train_state['test_acc'] = running_acc
    
    print("Test loss: {:.3f}".format(train_state['test_loss']))
    print("Test Accuracy: {:.2f}".format(train_state['test_acc']))

In [10]:
args = Namespace(
    # Data and path information
    frequency_cutoff=25,
    model_state_file='task1-model.pth',
    name='task1',
    save_dir='./model_storage/task1/',
    vectorizer_file='vectorizer.json',
    # No model hyperparameters
    # Training hyperparameters
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.01,
    num_epochs=5,
    seed=1337,
    # Runtime options
    cuda=True,
    device='cuda',
    vectoriser=PretrainedVectoriser,
    data_field='tfidf10',
    feature_field="is_fulltime",
    is_sequence=False,
    sent_embed=True
)

In [27]:
do_FWD_train(args)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Test loss: 0.548
Test Accuracy: 73.26


In [26]:

def make_train_state(args):
    return {'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': 1,
            'test_acc': 1}

train_state = make_train_state(args)
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")

# dataset and vectorizer
# dataset2 = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.input_name, Word2vecVectoriser)
# vectorizer2 = dataset2.get_vectorizer()
dataset = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.name, OneHotVectoriser, 'tfidf10', 'is_fulltime', False, True)
vectorizer = dataset.get_vectorizer()
# model
# classifier = JobDescriptionClassifier(num_features=len(vectorizer.input_vocab))
classifier = JobDescriptionNotSimpleClassifier(num_features=vectorizer.num_features)
classifier = classifier.to(args.device)
# loss and optimizer
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)

def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()#.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100




#### Training Loop

In [27]:
for epoch_index in range(args.num_epochs):
    train_state['epoch_index'] = epoch_index
    # Iterate over training dataset
    # setup: batch generator, set loss and acc to 0, set train mode on
    dataset.set_split('train')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.0
    running_acc = 0.0
    classifier.train()
    for batch_index, batch_dict in tqdm(enumerate(batch_generator), colour='white'):
        # the training routine is 5 steps:
        # step 1. zero the gradients
        optimizer.zero_grad()
        # step 2. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 3. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch-running_loss) / (batch_index + 1)
        # step 4. use loss to produce gradients
        loss.backward()
        # step 5. use optimizer to take gradient step
        optimizer.step()
        # compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)

    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)

    # Iterate over val dataset
    # setup: batch generator, set loss and acc to 0, set eval mode on
    dataset.set_split('val')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):
        # step 1. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 2. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch - running_loss) / (batch_index + 1)
        # step 3. compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)
    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

KeyboardInterrupt: 

#### Evaluation

In [21]:
dataset.set_split('test')
batch_generator = generate_batches(dataset,batch_size=args.batch_size,device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()
for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred = classifier(x_in=batch_dict['x_data'].float())
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_batch = loss.item()
    running_loss += (loss_batch - running_loss) / (batch_index + 1)
    # compute the accuracy
    acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_batch - running_acc) / (batch_index + 1)
train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [22]:
print("Test loss: {:.3f}".format(train_state['test_loss']))
print("Test Accuracy: {:.2f}".format(train_state['test_acc']))

Test loss: 0.573
Test Accuracy: 68.63


### CNN Conv1D Model

In [11]:
class CNNEmbedClassifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings, num_channels, 
                 hidden_dim, num_classes, dropout_p, 
                 pretrained_embeddings=None, padding_idx=0):
        """
        Args:
            embedding_size (int): size of the embedding vectors
            num_embeddings (int): number of embedding vectors
            filter_width (int): width of the convolutional kernels
            num_channels (int): number of convolutional kernels per layer
            hidden_dim (int): the size of the hidden dimension
            num_classes (int): the number of classes in classification
            dropout_p (float): a dropout parameter 
            pretrained_embeddings (numpy.array): previously trained word embeddings
                default is None. If provided, 
            padding_idx (int): an index representing a null position
        """
        super(CNNEmbedClassifier, self).__init__()

        if pretrained_embeddings is None:

            self.emb = nn.Embedding(embedding_dim=embedding_size,
                                    num_embeddings=num_embeddings,
                                    padding_idx=padding_idx)        
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.emb = nn.Embedding(embedding_dim=embedding_size,
                                    num_embeddings=num_embeddings,
                                    padding_idx=padding_idx,
                                    _weight=pretrained_embeddings)
        
            
        self.convnet = nn.Sequential(
            nn.Conv1d(in_channels=embedding_size, 
                   out_channels=num_channels, kernel_size=1),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=2, stride=2),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=2, stride=2),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=2),
            nn.ELU()
        )

        self._dropout_p = dropout_p
        self.fc1 = nn.Linear(num_channels, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x_in, apply_softmax=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, dataset._max_seq_length)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, num_classes)
        """
        # embed and permute so features are channels
        x_embedded = self.emb(x_in).permute(0, 2, 1)

        features = self.convnet(x_embedded)
        print(features.shape)
        print(features)
        # average and remove the extra dimension
        remaining_size = features.size(dim=2)
        features = F.avg_pool1d(features, remaining_size).squeeze(dim=2)
        features = F.dropout(features, p=self._dropout_p)
        
        # mlp classifier
        intermediate_vector = F.relu(F.dropout(self.fc1(features), p=self._dropout_p))
        prediction_vector = self.fc2(intermediate_vector)

        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)

        return prediction_vector


In [21]:
class CNNClassifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings, num_channels, 
                 hidden_dim, num_classes, dropout_p, 
                 pretrained_embeddings=None, padding_idx=0):
        """
        Args:
            embedding_size (int): size of the embedding vectors
            num_embeddings (int): number of embedding vectors
            filter_width (int): width of the convolutional kernels
            num_channels (int): number of convolutional kernels per layer
            hidden_dim (int): the size of the hidden dimension
            num_classes (int): the number of classes in classification
            dropout_p (float): a dropout parameter 
            pretrained_embeddings (numpy.array): previously trained word embeddings
                default is None. If provided, 
            padding_idx (int): an index representing a null position
        """
        super(CNNClassifier, self).__init__()
        
            
        self.convnet = nn.Sequential(
            nn.Conv1d(in_channels=embedding_size, 
                   out_channels=num_channels, kernel_size=1),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=2, stride=2),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=2, stride=2),
            nn.ELU(),
            nn.Conv1d(in_channels=num_channels, out_channels=num_channels, 
                   kernel_size=2),
            nn.ELU()
        )

        self._dropout_p = dropout_p
        self.fc1 = nn.Linear(num_channels, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x_in, apply_softmax=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, dataset._max_seq_length)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, num_classes)
        """
        # embed and permute so features are channels
#         x_embedded = self.emb(x_in).permute(0, 2, 1)

        features = self.convnet(x_in)
        print(features.shape)
        print(features)
        # average and remove the extra dimension
#         remaining_size = features.size(dim=2)
#         features = F.avg_pool1d(features, remaining_size).squeeze(dim=2)
#         features = F.avg_pool1d(features, remaining_size).squeeze(dim=2)
        features = F.dropout(features, p=self._dropout_p)
        
        # mlp classifier
        intermediate_vector = F.relu(F.dropout(self.fc1(features), p=self._dropout_p))
        prediction_vector = self.fc2(intermediate_vector)

        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)

        return prediction_vector

#### Train Setup

In [53]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

## Do Things

In [14]:


def do_train(args):
    if args.expand_filepaths_to_save_dir:
        args.vectorizer_file = os.path.join(args.save_dir,
                                            args.vectorizer_file)

        args.model_state_file = os.path.join(args.save_dir,
                                             args.model_state_file)

        print("Expanded filepaths: ")
        print("\t{}".format(args.vectorizer_file))
        print("\t{}".format(args.model_state_file))

    # Check CUDA
    if not torch.cuda.is_available():
        args.cuda = False

    args.device = torch.device("cuda" if args.cuda else "cpu")
    print("Using CUDA: {}".format(args.cuda))

    # Set seed for reproducibility
    set_seed_everywhere(args.seed, args.cuda)

    # handle dirs
    handle_dirs(args.save_dir)
    # args.use_glove = True

    if args.reload_from_files:
        # training from a checkpoint
        dataset = NewsDataset.load_dataset_and_load_vectorizer(args.news_csv,
                                                               args.vectorizer_file)
    else:
        dataset = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.name, args.vectoriser,  args.data_field, args.feature_field, 
            args.is_sequence, args.sent_embed)
        # create dataset and vectorizer
        # dataset = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.name, vectoriser=CNNVectorizer)
        # dataset.save_vectorizer(args.vectorizer_file)
    vectorizer = dataset.get_vectorizer()

    # Use GloVe or randomly initialized embeddings
    if args.use_glove:
        words = vectorizer.input_vocab._token_to_idx.keys()
        embeddings = make_embedding_matrix(glove_filepath=args.glove_filepath, 
                                           words=words)
        print("Using pre-trained embeddings")
    else:
        print("Not using pre-trained embeddings")
        embeddings = None


    classifier = args.classifier(embedding_size=args.embedding_size, 
                                num_embeddings=len(vectorizer.input_vocab),
                                num_channels=args.num_channels,
                                hidden_dim=args.hidden_dim, 
                                num_classes=len(vectorizer.class_vocab), 
                                dropout_p=args.dropout_p,
                                pretrained_embeddings=embeddings,
                                padding_idx=0)
    
    classifier = classifier.to(args.device)
    dataset.class_weights = dataset.class_weights.to(args.device)

    loss_func = nn.CrossEntropyLoss(dataset.class_weights)
    optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                               mode='min', factor=0.5,
                                               patience=1)

    train_state = make_train_state(args)

    epoch_bar = tqdm(desc='training routine', 
                              total=args.num_epochs,
                              position=0)

    dataset.set_split('train')
    train_bar = tqdm(desc='split=train',
                              total=dataset.get_num_batches(args.batch_size), 
                              position=1, 
                              leave=True)
    dataset.set_split('val')
    val_bar = tqdm(desc='split=val',
                            total=dataset.get_num_batches(args.batch_size), 
                            position=1, 
                            leave=True)


    for epoch_index in range(args.num_epochs):
    # for epoch_index in range(2):
        train_state['epoch_index'] = epoch_index
        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on

        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()
    #         print(batch_dict['x_data'])
            # step 2. compute the output
            y_pred = classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss, acc=running_acc, 
                                  epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred =  classifier(batch_dict['x_data'])

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                            epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        if train_state['stop_early']:
            break

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()
        
    # compute the loss & accuracy on the test set using the best available model

    classifier.load_state_dict(torch.load(train_state['model_filename']))

    classifier = classifier.to(args.device)
    dataset.class_weights = dataset.class_weights.to(args.device)
    loss_func = nn.CrossEntropyLoss(dataset.class_weights)

    dataset.set_split('test')
    batch_generator = generate_batches(dataset, 
                                       batch_size=args.batch_size, 
                                       device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):
        # compute the output
        y_pred =  classifier(batch_dict['x_data'])

        # compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'])
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        # compute the accuracy
        acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_t - running_acc) / (batch_index + 1)

    train_state['test_loss'] = running_loss
    train_state['test_acc'] = running_acc

    print("Test loss: {};".format(train_state['test_loss']))
    print("Test Accuracy: {}".format(train_state['test_acc']))



## One hot embeddings

In [17]:
args = Namespace(
    # Data and Path hyper parameters
    name="task1",
    news_csv="./data/ag_news/news_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="./model_storage/document_classification",
    # Model hyper parameters
    glove_filepath='/home/dadams/Files/uni/nlp/data/glove.6B/glove.6B.100d.txt', 
    use_glove=False,
    embedding_size=100, 
    hidden_dim=100, 
    num_channels=100, 
    # Training hyper parameter
    seed=1337, 
    learning_rate=0.001, 
    dropout_p=0.1, 
    batch_size=128, 
    num_epochs=1, 
    early_stopping_criteria=5, 
    # Runtime option
    cuda=True, 
    catch_keyboard_interrupt=True, 
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
    # Dataset params
    data_field='tfidf10', 
    feature_field='is_fulltime', 
    is_sequence=True, 
    sent_embed=True,
    vectoriser=OneHotVectoriser,
    classifier=CNNEmbedClassifier
)

In [18]:
do_train(args)

Expanded filepaths: 
	./model_storage/document_classification/vectorizer.json
	./model_storage/document_classification/model.pth
Using CUDA: True
Not using pre-trained embeddings


training routine:   0%|          | 0/1 [00:00<?, ?it/s]

split=train:   0%|          | 0/96 [00:00<?, ?it/s]

split=val:   0%|          | 0/27 [00:00<?, ?it/s]

torch.Size([128, 100, 2])
tensor([[[-0.1472,  0.0394],
         [ 0.1039, -0.0013],
         [-0.1122, -0.0957],
         ...,
         [ 0.0170,  0.0397],
         [-0.0994, -0.0693],
         [-0.0548, -0.0428]],

        [[ 0.1046,  0.0522],
         [ 0.0280,  0.0840],
         [-0.1830, -0.1032],
         ...,
         [-0.0555,  0.0628],
         [-0.0061, -0.0548],
         [-0.0741,  0.0919]],

        [[ 0.0707, -0.0291],
         [ 0.0471, -0.1088],
         [-0.1642, -0.0890],
         ...,
         [ 0.0057,  0.0902],
         [-0.1363, -0.0962],
         [ 0.0142,  0.0192]],

        ...,

        [[ 0.0361,  0.0366],
         [ 0.0256,  0.1090],
         [-0.1092, -0.0502],
         ...,
         [ 0.1017,  0.0280],
         [ 0.0756, -0.1529],
         [-0.1127,  0.0224]],

        [[ 0.0042,  0.0141],
         [ 0.0155,  0.0256],
         [-0.1138, -0.0795],
         ...,
         [ 0.0484,  0.1205],
         [-0.0264, -0.0180],
         [ 0.0330, -0.0263]],

        [[

...

torch.Size([128, 100, 2])
tensor([[[-0.0942,  0.1576],
         [-0.0563,  0.1273],
         [-0.3790, -0.1537],
         ...,
         [-0.1870,  0.0599],
         [ 0.3490,  0.3209],
         [-0.3422, -0.1483]],

        [[-0.1852, -0.0382],
         [ 0.0080,  0.0553],
         [-0.2479,  0.3864],
         ...,
         [ 0.1672,  0.3187],
         [-0.1527, -0.0886],
         [ 0.1411,  0.0985]],

        [[-0.1148,  0.2793],
         [-0.0721,  0.2181],
         [-0.3923, -0.4476],
         ...,
         [-0.0470, -0.3140],
         [ 0.0172,  0.6204],
         [-0.1211, -0.1604]],

        ...,

        [[ 0.1048, -0.0235],
         [-0.0471, -0.0275],
         [-0.1802,  0.3992],
         ...,
         [-0.1576,  0.3444],
         [ 0.2639, -0.1146],
         [-0.2349,  0.2491]],

        [[-0.3022,  0.2224],
         [-0.0290,  0.0222],
         [-0.3785, -0.2187],
         ...,
         [ 0.0478, -0.1626],
         [-0.3912,  0.2883],
         [ 0.3961, -0.2904]],

        [[

tensor([[[-0.1646, -0.1079],
         [ 0.0627, -0.0248],
         [ 0.0717,  0.4682],
         ...,
         [ 0.1296,  0.4172],
         [-0.1741, -0.2189],
         [ 0.2099,  0.2979]],

        [[-0.1699,  0.2253],
         [ 0.0678, -0.0462],
         [-0.5486, -0.4465],
         ...,
         [-0.3440, -0.2379],
         [ 0.1387,  0.4214],
         [-0.1726, -0.4738]],

        [[ 0.0408, -0.0527],
         [-0.0903,  0.0627],
         [-0.0650,  0.1700],
         ...,
         [ 0.1143,  0.2514],
         [ 0.3190, -0.1397],
         [-0.2924,  0.1828]],

        ...,

        [[-0.1784, -0.1363],
         [-0.0505,  0.0729],
         [ 0.3944, -0.5104],
         ...,
         [ 0.2290, -0.2128],
         [-0.1309,  0.0978],
         [ 0.2594, -0.0044]],

        [[-0.0720,  0.1543],
         [ 0.1011, -0.2597],
         [-0.7472,  0.7166],
         ...,
         [-0.5638,  0.5894],
         [ 0.7115,  0.1705],
         [-0.5008, -0.1565]],

        [[ 0.0711,  0.2932],
       

torch.Size([128, 100, 2])
tensor([[[ 0.4613,  0.0078],
         [ 0.2105, -0.0303],
         [-0.7311,  0.0567],
         ...,
         [-0.5210, -0.0834],
         [ 0.9198,  0.1230],
         [-0.5776, -0.0739]],

        [[-0.2295,  0.1587],
         [ 0.1380,  0.3416],
         [-0.0772, -0.6183],
         ...,
         [-0.0313, -0.5282],
         [-0.2213,  0.4887],
         [ 0.2667, -0.3115]],

        [[-0.1223, -0.1174],
         [-0.3634,  0.2857],
         [ 0.9823, -0.0171],
         ...,
         [ 0.9596,  0.1758],
         [ 0.0211, -0.3359],
         [ 0.0027,  0.5676]],

        ...,

        [[ 0.2402,  0.2890],
         [-0.1399,  0.1082],
         [-0.2102, -0.5113],
         ...,
         [-0.1851, -0.2702],
         [ 0.1274,  0.2778],
         [-0.3754, -0.3002]],

        [[ 0.0218,  0.0309],
         [ 0.3355, -0.1203],
         [-0.7375,  0.4862],
         ...,
         [-0.4372,  0.1093],
         [ 0.4771,  0.0424],
         [-0.2524, -0.0612]],

        [[

torch.Size([128, 100, 2])
tensor([[[-0.0973, -0.0684],
         [-0.0551,  0.0329],
         [-0.0104,  0.4746],
         ...,
         [ 0.1263,  0.3957],
         [-0.0952, -0.1409],
         [ 0.1218,  0.2820]],

        [[ 0.1432, -0.1264],
         [-0.1116, -0.2181],
         [-0.2862,  0.8665],
         ...,
         [-0.0846,  0.6004],
         [ 0.4534, -0.2959],
         [-0.3908,  0.3247]],

        [[-0.0651,  0.3312],
         [ 0.2233, -0.0248],
         [-0.5425, -0.6182],
         ...,
         [-0.3158, -0.3595],
         [ 0.2532,  0.4032],
         [-0.1439, -0.4573]],

        ...,

        [[-0.1617,  0.0348],
         [-0.1041,  0.1251],
         [-0.0813, -0.1304],
         ...,
         [ 0.0716,  0.0887],
         [ 0.0906, -0.0411],
         [-0.2160, -0.0913]],

        [[-0.0548, -0.1581],
         [-0.3117, -0.0109],
         [ 1.0582,  0.4723],
         ...,
         [ 0.7214,  0.5228],
         [-0.1278, -0.5461],
         [ 0.1766,  0.7466]],

        [[

torch.Size([128, 100, 2])
tensor([[[ 0.0861,  0.1123],
         [-0.2424,  0.1602],
         [ 0.1434, -0.2640],
         ...,
         [ 0.2931, -0.1969],
         [-0.1490,  0.2509],
         [-0.1889,  0.1879]],

        [[-0.2557, -0.1196],
         [-0.2224,  0.3563],
         [ 0.8166, -0.6401],
         ...,
         [ 0.5559, -0.3422],
         [-0.3223,  0.2181],
         [ 0.2977, -0.0340]],

        [[-0.1167, -0.1734],
         [ 0.0153, -0.2048],
         [ 0.1305,  0.6407],
         ...,
         [ 0.1126,  0.5663],
         [-0.1050, -0.3592],
         [-0.0241,  0.4073]],

        ...,

        [[ 0.0550,  0.0514],
         [-0.0134, -0.0208],
         [-0.4081,  0.5015],
         ...,
         [-0.1390,  0.3529],
         [ 0.2099, -0.1834],
         [-0.1835, -0.0063]],

        [[-0.0611, -0.0148],
         [ 0.0055, -0.1595],
         [-0.1848,  0.8967],
         ...,
         [ 0.0334,  0.4981],
         [-0.1790, -0.3513],
         [-0.0951,  0.3230]],

        [[

## Other Embeddings

In [22]:
args = Namespace(
    # Data and Path hyper parameters
    name="task1",
    news_csv="./data/ag_news/news_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="./model_storage/document_classification",
    # Model hyper parameters
    glove_filepath='/home/dadams/Files/uni/nlp/data/glove.6B/glove.6B.100d.txt', 
    use_glove=False,
    embedding_size=100, 
    hidden_dim=100, 
    num_channels=100, 
    # Training hyper parameter
    seed=1337, 
    learning_rate=0.001, 
    dropout_p=0.1, 
    batch_size=100, 
    num_epochs=5, 
    early_stopping_criteria=5, 
    # Runtime option
    cuda=True, 
    catch_keyboard_interrupt=True, 
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
    # Dataset params
    data_field='tfidf10', 
    feature_field='is_fulltime', 
    is_sequence=False, 
    sent_embed=False,
    vectoriser=Word2vecVectoriser,
    classifier=CNNClassifier
)

In [23]:
do_train(args)

Expanded filepaths: 
	./model_storage/document_classification/vectorizer.json
	./model_storage/document_classification/model.pth
Using CUDA: True
Time to build vocab: 0.01 mins
Time to train : 0.27 mins
Not using pre-trained embeddings


training routine:   0%|          | 0/5 [00:00<?, ?it/s]

split=train:   0%|          | 0/124 [00:00<?, ?it/s]

split=val:   0%|          | 0/35 [00:00<?, ?it/s]

torch.Size([100, 249])
tensor([[ 0.0312, -0.0551, -0.0602,  ..., -0.0124, -0.0017,  0.0412],
        [-0.0114, -0.0731,  0.0051,  ..., -0.1052,  0.0019, -0.0179],
        [ 0.0422, -0.0185, -0.0659,  ...,  0.0441,  0.0574,  0.0055],
        ...,
        [ 0.0883,  0.0048,  0.0635,  ...,  0.1124,  0.1873,  0.1266],
        [-0.0131,  0.0472, -0.0192,  ..., -0.0292, -0.0205, -0.0146],
        [-0.0990, -0.0687, -0.0527,  ..., -0.0195, -0.0870,  0.0148]],
       device='cuda:0', grad_fn=<EluBackward0>)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x249 and 100x100)

#### Setup 

In [52]:
if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)

    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# handle dirs
handle_dirs(args.save_dir)
# args.use_glove = True

if args.reload_from_files:
    # training from a checkpoint
    dataset = NewsDataset.load_dataset_and_load_vectorizer(args.news_csv,
                                                           args.vectorizer_file)
else:
    dataset = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.name, args.vectoriser, args.data_field, args.feature_field, 
        args.is_sequence, args.sent_embed)
    # create dataset and vectorizer
    # dataset = JobDescriptionDataset.load_dataset_and_make_vectorizer(args.name, vectoriser=CNNVectorizer)
    # dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.get_vectorizer()

# Use GloVe or randomly initialized embeddings
if args.use_glove:
    words = vectorizer.input_vocab._token_to_idx.keys()
    embeddings = make_embedding_matrix(glove_filepath=args.glove_filepath, 
                                       words=words)
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None


print(len(vectorizer.class_vocab))

classifier = CNNClassifier(embedding_size=args.embedding_size, 
                            num_embeddings=len(vectorizer.input_vocab),
                            num_channels=args.num_channels,
                            hidden_dim=args.hidden_dim, 
                            num_classes=len(vectorizer.class_vocab), 
                            dropout_p=args.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0)

Expanded filepaths: 
	./model_storage/document_classification/./model_storage/document_classification/./model_storage/document_classification/vectorizer.json
	./model_storage/document_classification/./model_storage/document_classification/./model_storage/document_classification/model.pth
Using CUDA: True
Not using pre-trained embeddings
2


#### Train

In [60]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
    
loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                           mode='min', factor=0.5,
                                           patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm(desc='training routine', 
                          total=args.num_epochs,
                          position=0)

dataset.set_split('train')
train_bar = tqdm(desc='split=train',
                          total=dataset.get_num_batches(args.batch_size), 
                          position=1, 
                          leave=True)
dataset.set_split('val')
val_bar = tqdm(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1, 
                        leave=True)


for epoch_index in range(args.num_epochs):
# for epoch_index in range(2):
    train_state['epoch_index'] = epoch_index
    # Iterate over training dataset

    # setup: batch generator, set loss and acc to 0, set train mode on

    dataset.set_split('train')
    batch_generator = generate_batches(dataset, 
                                       batch_size=args.batch_size, 
                                       device=args.device)
    running_loss = 0.0
    running_acc = 0.0
    classifier.train()

    for batch_index, batch_dict in enumerate(batch_generator):
        # the training routine is these 5 steps:

        # --------------------------------------
        # step 1. zero the gradients
        optimizer.zero_grad()
#         print(batch_dict['x_data'])
        # step 2. compute the output
        y_pred = classifier(batch_dict['x_data'])
        
        # step 3. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'])
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        # step 4. use loss to produce gradients
        loss.backward()

        # step 5. use optimizer to take gradient step
        optimizer.step()
        # -----------------------------------------
        # compute the accuracy
        acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_t - running_acc) / (batch_index + 1)

        # update bar
        train_bar.set_postfix(loss=running_loss, acc=running_acc, 
                              epoch=epoch_index)
        train_bar.update()

    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)

    # Iterate over val dataset

    # setup: batch generator, set loss and acc to 0; set eval mode on
    dataset.set_split('val')
    batch_generator = generate_batches(dataset, 
                                       batch_size=args.batch_size, 
                                       device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):

        # compute the output
        y_pred =  classifier(batch_dict['x_data'])

        # step 3. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'])
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        # compute the accuracy
        acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_t - running_acc) / (batch_index + 1)
        val_bar.set_postfix(loss=running_loss, acc=running_acc, 
                        epoch=epoch_index)
        val_bar.update()

    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)

    train_state = update_train_state(args=args, model=classifier,
                                     train_state=train_state)

    scheduler.step(train_state['val_loss'][-1])

    if train_state['stop_early']:
        break

    train_bar.n = 0
    val_bar.n = 0
    epoch_bar.update()
    


training routine:   0%|          | 0/5 [00:00<?, ?it/s]

split=train:   0%|          | 0/96 [00:00<?, ?it/s]

split=val:   0%|          | 0/27 [00:00<?, ?it/s]

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.DoubleTensor instead (while checking arguments for embedding)

#### Evaluate

In [ ]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred =  classifier(batch_dict['x_data'])
    
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [ ]:
print("Test loss: {};".format(train_state['test_loss']))
print("Test Accuracy: {}".format(train_state['test_acc']))

### CNN Conv1D (full job description)

In [ ]:
args = Namespace(
    # Data and Path hyper parameters
    name="task1",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/document_classification",
    # Model hyper parameters
    glove_filepath='/home/dadams/Files/uni/nlp/data/glove.6B/glove.6B.100d.txt', 
    use_glove=False,
    embedding_size=100, 
    hidden_dim=100, 
    num_channels=100, 
    # Training hyper parameter
    seed=1337, 
    learning_rate=0.001, 
    dropout_p=0.1, 
    batch_size=128, 
    num_epochs=20, 
    early_stopping_criteria=5, 
    # Runtime option
    cuda=True, 
    catch_keyboard_interrupt=True, 
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
    # Dataset params
    data_field='job_description', 
    feature_field='is_fulltime', 
    is_sequence=True, 
    sent_embed=True,
    vectoriser=OneHotVectoriser,
    classifier=CNNEmbedClassifier
) 

In [ ]:
do_train(args)

## Comparison / Discussion